In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [ ]:
from asyncio import run

from asyncssh import connect


# TODO
async def hpc():
    url = 'hpc.computing.eu'
    username = 'your_user'

    async with connect(url, username=username) as connection:
        result = await connection.run('qstat', check=True)
        stdout = result.stdout.strip()

        print(result)

        return stdout


run(hpc())